# Matmul

$
\begin{align*}
\overset{ (N,M) }{ Y } 
&= 
\begin{bmatrix}
{ Y_{(n=0)} } \\
\vdots \\
{ Y_{(n)} } \\
\vdots \\
{ Y_{(n=N-1)} }
\end{bmatrix}
=
\overset{ (N,D) }{ X } \; @ \; \overset{ (D,M) }{ W^T }
\\
\\
\overset{ (M,) }{ Y_{(n)} } &= (y_{(n)(m=0)}, \; \dots, \; y_{(n)(m)},  \; \dots, \; y_{(n)(m=M-1)})
\\
\overset{ () }{ y_{(n)(m)} }
&= \overset{ (D,) }{ X_{(n)} } \cdot \overset{ (D,) }{ W_{(m)}^T }
= \sum\limits ^{D}_{d=0}  \overset{ () }{ x_{(n)(d)} } * \overset{ () }{ w_{(m)(d)} }
\\
_{(0 \le d \le D, \; 0 \le n \lt N, \; 0 \le m \lt M)}
\end{align*}
$

# Activation

$
\begin{align*}
\overset{ (N,M) }{ A } &= 
activation \left( 
    \overset{ (N,M) }{ Y }  = 
    \begin{bmatrix}
    { Y_{(n=0)} } \\
    \vdots \\
    { Y_{(n)} } \\
    \vdots \\
    { Y_{(n=N-1)} }
    \end{bmatrix}
\right)
\\
\overset{ (M,) }{ A_{(n)} } 
&= activation \left( \overset{ (M,) }{ Y_{(n) }} \right)  \\
&= (a_{(n)(m=0)}, \; \dots, \; a_{(n)(m)},  \; \dots, \; a_{(n)(m=M-1)})
\\
\overset{ () }{ a_{(n)(m)} } &= activation \left( \overset{ () }{ y_{(n)(m)} } \right)
\quad _{(0 \le n \lt N, \; 0 \le m \lt M)}
\end{align*}
$

# Softmax
$C_n$ is to prevent the overflow at $np.exp()$.

```
def softmax(x):
    """Softmax function from deep-learning-from-scratch
    Args:
        x: batch input data of shape (N x M).
            N: Batch size
            M: Number of nodes
    Returns:
        Prediction probability matrix P of shape (N x M)
    """
    c = np.max(x, axis=-1, keepdims=True)   # オーバーフロー対策
    exp = np.exp(x - c)
    return exp / np.sum(exp, axis=-1, keepdims=True)
```

<img src="images/SMLL.png" align="left" />

$
\begin{align*}
\overset{ (N,1) }{ C } &= np.max\left( 
    \overset{ (N,M) }{ A }, \; axis=-1,  \; keepdim=True 
\right)
\\
\overset{ (N,M) }{ EXP } &= np.exp \left( \overset{ (N,M) }{ A } - \overset{ (N,1) }{ C } \right)
= np.exp \left(
    \begin{bmatrix}
    { A_{(n=0)} } - { C_{(n=0)} }\\
    \vdots \\
    { A_{(n)} }   - { C_{(n)} }\\
    \vdots \\
    { A_{(n=N-1)} } - { C_{(n=N-1)} }\\
    \end{bmatrix}
\right) 
\\
\overset{ (N,1) }{ sum(EXP) } &= np.sum \left( 
    \overset{ (N,M) }{ EXP }, \; axis=-1,  \; keepdim=True 
\right)
\\
\overset{ (N,M) }{ P } &= \overset{ (N,M) }{ EXP }  \;\; / \;\; \overset{ (N,1) }{ sum(EXP) } 
\\
\overset{ (N,) }{ P_{(n)} } &= (p_{(n)(m=0)}, \; \dots, \; p_{(n)(m)} , \; \dots, \; p_{(n)(m=M-1)})
\\
{ p_{(n)(m)} } 
&= \frac {np.exp \left( 
    { a_{(n)(m) } } - { c_{(n)(m)} }) \right) 
}
{  
np.sum \left( 
    np.exp \left( 
        a_{(n)(m) } - c_{(n)(m)}
    \right)
\right) 
}
\end{align*}
$

# Labels

## For One Hot Encoding labels
$
\begin{align*}
\overset{ (N,M) }{ T_{_{OHE}} } &= ( 
    \overset{ (M,) }{ T_{(n=0)} }, \; \dots \;, \overset{ (M,) }{ T_{(n=N-1)} } 
) \qquad \text { for one hot encoding labels} 
\\
\overset{ (M,) }{ T_{ _{OHE} (n)} } &= ( \overset{ () }{ t_{(n)(m=0)} }, \; \dots \;, \overset{ () }{ t_{(n)(m=M-1)} })
\end{align*}
$

## For index labels
$
\begin{align*}
\overset{ (N,) }{ T_{_{IDX}} } &= (\overset{ () }{ t_{(n=0)} }, \; \dots \;, \overset{ () }{ t_{(n=N-1)} }) \qquad \text {for index labels }
\end{align*}
$

# Cross Entropy Log Loss

```
def cross_entropy_error(p, t):
    """Cross entropy log loss for multi labels.
    NOTE:
        Handle only the label whose value is True. The reason not to use non-labels to
        calculate the loss is TBD.

    Args:
        p: probability matrix of shape (N x M) from soft-max layer where:
            N is Batch size
            M is Number of nodes
        t: label either One Hot Encoding (OHE) format or indexing format.
    Returns:
        L: Loss value normalized by the batch size N, hence a scalar value.
    """
    if p.ndim == 1:
        t = t.reshape(1, t.size)
        p = p.reshape(1, p.size)
    N = batch_size = p.shape[0]

    # --------------------------------------------------------------------------------
    # Convert into index labels format from OHE ones.
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    # --------------------------------------------------------------------------------
    if t.size == p.size:
        t = t.argmax(axis=1)

    # --------------------------------------------------------------------------------
    # Logg loss with a small value e added to prevent infinitive value by log(+e).
    # Use numpy tuple indexing to select e.g. P[n=0][m=2] and P[n=3][m=4] via:
    # P[
    #   (0, 3),
    #   (2, 4)
    # ]
    # --------------------------------------------------------------------------------
    e = 1e-7
    return -np.sum(np.log(p[np.arange(batch_size), t] + e)) / N
```

## For label using indexing 
For instance, if the multi labels are (0,1,2,3,4) then the index is 2 for the label 2. If the labels are (2,4,6,8,9), then the index is 3 for the label 8.  

Use LP to select the probabilities from P for the corresponding labels. For instance, if the label is 2 (hence the index is 2) for X(n=0), and 4 for X(n=3), then the numpy tuple indexing selects ```P[n=0][m=2]``` and ```P[n=3][m=4] ```.

```
P[
   (0, 3),
   (2, 4)
]
```

$
\text{ for index labels e.g. (5, 2, 0, 9, ...)}
\\
\\
\overset{ (N,) }{ J } = - np.sum(\; np.log(LP), \; axis = -1 \;) \\
LP = label\_probability = P \left[ \\
\quad ( \; 0, \; \dots, \;  {N-1}) , \\
\quad ( \; t_{(n=0)} \; , \dots , \; t_{(n=N-1)}) \\
\right]
\\
\\
$

## For label using One Hot Encoding (OHE)
For instance, if multi labels are (0,1,2,3,4) and each label is OHE, then the label for 2 is (0,0,1,0,0).   
<br>
$
\text{ for one hot encoding labels }
\\
\overset{ (N,) }{ J } = - \sum\limits^{M-1}_{m=0} 
    \left[ \; \;  
        t_{(n)(m)} \;  * \;  np.log(p_{(n)(m)}) \;\;  
    \right]
\\
\overset{ () }{ J_{(n)} } = \overset{ (M,) }{ T_{(n)} } \cdot \overset{ (M,) }{ P_{(n)} } 
$
  
## Total loss of a batch


$
\overset{ () }{ L } = \frac {1}{N} \sum\limits^{N-1}_{n=0} \overset{ () }{ J_{{(n)}} }
$

## Misc
$
\\
\sum\limits^{M-1}_{m=0}
\\
\frac {1}{N} \sum\limits^{N-1}_{n=0} 
$

In [2]:
import numpy as np

In [12]:
a = np.arange(6).reshape(2,3)
print(a)
b = np.arange(0,-6,-1).reshape(2,3)
print(b.T)
c = [
    np.inner(a[0], b[0]),
    np.inner(a[1], b[1]),    
]
print(c)

[[0 1 2]
 [3 4 5]]
[[ 0 -3]
 [-1 -4]
 [-2 -5]]
[-5, -50]
